In [1]:
import sys
for entry in sys.path:
    if 'python2.7' in entry:
        sys.path.remove(entry)

In [2]:
sys.path.remove('/usr/local/lib/python2.7/site-packages/mxnet-0.9.1-py2.7-macosx-10.11-x86_64.egg')

In [3]:
sys.path

['',
 '/Applications/anaconda3/lib/python36.zip',
 '/Applications/anaconda3/lib/python3.6',
 '/Applications/anaconda3/lib/python3.6/lib-dynload',
 '/Users/khanh94/.local/lib/python3.6/site-packages',
 '/Applications/anaconda3/lib/python3.6/site-packages',
 '/Applications/anaconda3/lib/python3.6/site-packages/aeosa',
 '/Applications/anaconda3/lib/python3.6/site-packages/torchvision-0.1.9-py3.6.egg',
 '/Users/khanh94/.ipython']

In [1]:
import pickle
import numpy as np
import tensorflow as tf

#Loading the image files
data_file = open("cifar_10_tf_train_test.pkl", "rb")
train_x, train_y, test_x, test_y = pickle.load(data_file, encoding='latin1')
train_x = (train_x - np.mean(train_x)) / np.std(train_x)
test_x = (test_x - np.mean(test_x)) / np.std(test_x)
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

/Applications/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


FileNotFoundError: [Errno 2] No such file or directory: 'cifar_10_tf_train_test.pkl'

In [ ]:
#Getting the shape of the training tensors
train_y.shape
test_y.shape

In [ ]:
#Getting the shape of the testing tensors
test_x.shape
test_y.shape

In [ ]:
import tensorflow as tf

In [ ]:
#Wrapper functions to implement the layers
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [ ]:
#Placeholders
X = tf.placeholder(tf.float32, [None, 32, 32, 3], name='dataset')
Y_ = tf.placeholder(tf.int32, [None])
lr = tf.placeholder(tf.float32)

In [ ]:
#First convolutional layer with max pooling
w_conv1 = weight_variable([5, 5, 3, 32], 'weight1')
b_conv1 = bias_variable([32], 'bias1')
h_conv1 = tf.add(conv2d(X, w_conv1), b_conv1, name='pre_relu1')
h_conv1 = tf.nn.relu(h_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
h_pool1.shape

In [ ]:
#Second convolutional layer with max pooling
w_conv2 = weight_variable([5, 5, 32, 32], 'weight2')
b_conv2 = bias_variable([32], 'bias2')
h_conv2 = tf.add(conv2d(h_pool1, w_conv2), b_conv2, name='pre_relu2')
h_conv2 = tf.nn.relu(h_conv2)
h_conv2.shape
h_pool2 = max_pool_2x2(h_conv2)
h_pool2.shape

In [ ]:
#Third convolutional layer with max pooling
w_conv3 = weight_variable([3, 3, 32, 64], 'weight3')
b_conv3 = bias_variable([64], 'bias3')
h_conv3 = tf.add(conv2d(h_pool2, w_conv3), b_conv3, name='pre_relu3')

In [ ]:
#Fully connected layer with softmax
fully_connected = tf.contrib.layers.flatten(h_conv3)
final_weight = tf.Variable(tf.truncated_normal([576, 10], stddev=0.1), name='final_weight')
final_bias = tf.Variable(tf.constant(0.1, tf.float32, [10]), name='last_bias')
Y_logits = tf.matmul(fully_connected, final_weight) + final_bias
Y = tf.nn.softmax(Y_logits)

In [ ]:
Y.shape
# Y_.shape

In [ ]:
#Training the model with L2 regularization and Adam optimizer for backpropagation
beta = 0.001
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=Y_logits, labels=Y_)
print(cross_entropy)

cross_entropy = tf.reduce_mean(cross_entropy) + beta*tf.nn.l2_loss(w_conv1) + beta*tf.nn.l2_loss(b_conv1) + beta
*tf.nn.l2_loss(w_conv2) + beta*tf.nn.l2_loss(b_conv2) + beta*tf.nn.l2_loss(w_conv3)
+ beta*tf.nn.l2_loss(b_conv3) + beta*tf.nn.l2_loss(final_weight) + beta*tf.nn.l2_loss(final_bias)

print(cross_entropy)
# accuracy of the trained model, between 0 (worst) and 1 (best)

correct_prediction = tf.equal(tf.cast(tf.argmax(Y, 1), dtype=tf.int32), Y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
predictions = tf.argmax(Y, 1)

# training step, the learning rate is a placeholder
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

In [ ]:
accuracy

In [ ]:
#Get the relevant nodes and errors
import math
import numpy as np

tf.get_collection("validation_nodes")
# Add stuff to the collection.
tf.add_to_collection("validation_nodes", X)
tf.add_to_collection("validation_nodes", Y)
tf.add_to_collection("validation_nodes", Y_)
tf.add_to_collection("validation_nodes", predictions)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
training_epochs = 10000
learning_rate = 0.0008 

#Get training error and losses
training_error = []
testing_error = []
training_loss = []
testing_loss = []
for i in range(training_epochs):
    indices = np.random.randint(train_x.shape[0],size = 128)
    batch_X = train_x[indices]
    batch_Y = train_y[indices]
    # compute training values
    if i % 100 == 0:
        '''
        When we sess.run here, we are calculating the accuracy and cross_entropy of the model on batch_X and batch_Y (ie. on 100 pieces of data)
        '''
        a, c= sess.run([accuracy, cross_entropy], {X: batch_X, Y_: batch_Y})
        #temp = sess.run([correct_prediction], feed_dict={X: batch_X, Y_: batch_Y})
        print(str(i) + ": training accuracy:" + str(a) + " training loss: " + str(c) + " (lr:" + str(learning_rate) + ")")
        training_error.append(1 - a)
        training_loss.append(c)
        
    # compute test values
    if i % 200 == 0:
        '''
        When we sess.run here, we are calculating the accuracy and cross_entropy of the model on all of the data
        '''
        a, c= sess.run([accuracy, cross_entropy], {X: test_x, Y_: test_y})
        print(str(i) + ": test accuracy:" + str(a) + " test loss: " + str(c))
        testing_error.append(1 - a)
        testing_loss.append(c)
        
    # the backpropagation training step
    sess.run(train_step, {X: batch_X, Y_: batch_Y, lr: learning_rate})

    if i % 1000 == 0:
        saver = tf.train.Saver()
        save_path = saver.save(sess, "/Users/khanh94/Downloads/my_model_" + str(i))

In [ ]:
#Visualization for the filter
from PIL import Image
new_saver = tf.train.import_meta_graph('my_model_9000.meta')
new_saver.restore(sess,tf.train.latest_checkpoint('./'))

    # define layer name and number of filters
layer = 'pre_relu1'

    # start with a gray image with a little noise
img_noise = np.random.uniform(size=(1,32,32,3)) + 100.0
sess.run(tf.global_variables_initializer())

def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 1)*255)
    img = Image.fromarray(a, 'RGB')
    img.save('my.png')
    img.show()

def visstd(a, s=0.1):
    '''Normalize the image range for visualization'''
    return (a-a.mean())/max(a.std(), 1e-4)*s + 0.5

def T(layer):
    '''Helper for getting layer output tensor'''
    return tf.get_default_graph().get_tensor_by_name("{}:0".format(layer))

def render_naive(t_obj, img0=img_noise, iter_n=20, step=1.0):
    X = T('dataset')
    t_score = tf.reduce_mean(t_obj) # defining the optimization objective
    t_grad = tf.gradients(t_score, X)[0] # behold the power of automatic differentiation!

    img = img0.copy()
    for i in range(iter_n):
        g, score = sess.run([t_grad, t_score], {X:img})
        # normalizing the gradient, so the same step size should work 
        g /= g.std()+1e-8         # for different layers and networks
        img += g*step
    return np.squeeze(visstd(img), axis=0)

filter_imgs = []
for channel in range(32):
    filter_img = render_naive(T(layer)[:,:,:,channel])
    filter_imgs.append(filter_img)
filter_collage = [np.concatenate(filter_imgs[i:i+4], axis=0) for i in range(8)]
filter_collage = np.concatenate(filter_collage,axis=1)
filter_collage.shape
print("I passed ",filter_collage.shape)
showarray(filter_collage)
import matplotlib.pyplot as plt
plt.imshow('collage',filter_collage)
plt.waitKey(0)
plt.destroyAllWindows()

In [ ]:
#Plottting the error and loss curves
import matplotlib.pyplot as plt
plt.subplot(2,1,1)
plt.plot(training_error, '-o', label='Training_Error')
plt.title('Training Error')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend(ncol=2, loc='upper right')
plt.subplot(2,1,2)
plt.plot(testing_error, '-o', label='Testing_Error')
plt.title('Testing Error')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend(ncol=2, loc='lower right')
plt.gcf().set_size_inches(10, 10)
plt.savefig('Training_and_testing_error.png')

In [ ]:
#Plotting the training and testing losses
plt.subplot(2,1,1)
plt.plot(training_loss, '-o', label='Training_Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(ncol=2, loc='upper right')
plt.subplot(2,1,2)
plt.plot(testing_loss, '-o', label='Testing_Loss')
plt.title('Testing Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(ncol=2, loc='lower right')
plt.gcf().set_size_inches(10, 10)
plt.savefig('Training_and_testing_loss.png')

In [ ]:
indices = np.random.randint(train_x.shape[0],size = 50)

In [ ]:
indices

In [ ]:
train_x[indices,:].shape

In [ ]:
type(train_x)

In [ ]:
train_x.shape[0]

In [ ]:
np.array(train_x)[indices,:].shape

In [ ]:
train_y.shape

In [ ]:
np.array(train_y)[indices].shape

In [ ]:
test_y.shape

In [ ]:
batch_Y.shape

In [ ]:
train_y[indices].shape

In [ ]:
pwd